In [5]:
import os
from datetime import datetime
import pandas_datareader as pdr
from urllib import urlopen
from bs4 import BeautifulSoup
import json
import pandas as pd
import requests
import csv
from tqdm import tnrange, tqdm_notebook
from time import sleep
import pickle
import matplotlib.pyplot as plt
from pandas.tseries.offsets import BDay

%pylab inline


no_hist_symbol = []
DIR = 'C:/Users/sarab/Desktop/git_projects/stock_analyst_grade_history/'
URL = 'https://finance.yahoo.com/quote/{0}/analysts?p={0}'

Populating the interactive namespace from numpy and matplotlib


In [2]:
##############################################################################
#### Populating Ticker Lists #################################################
##############################################################################

def locate_all_symbols(suffix,base_filename,dir_name):
    script_dir = os.path.join(dir_name,'.'.join((base_filename,suffix))) 
    all_companies = pd.read_csv(script_dir)
    all_ticker_symbols = all_companies['Symbol'].tolist()
    return all_ticker_symbols

def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)      
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)    
    return tickers    


entire_ticker_list = locate_all_symbols('csv','company_ticker_symbols', DIR)
snp500tickers = save_sp500_tickers()

In [3]:
def select_ticker(tick):
    url = 'https://finance.yahoo.com/quote/{0}/analysts?p={0}'
    return url.format(tick)

def yahoo_finance_analyst_data(url):
    req = urlopen(url)
    raw = req.read()
    soup = BeautifulSoup(raw,'lxml')
    return soup

def auto_symbols(ticker):
    url = select_ticker(ticker)
    return url

def grab_dict(soup):
    l = [i for i in soup.find_all('script')]
    mylist = [str(item) for item in l]
    m = max(mylist,key=len)
    ind = mylist.index(m)
    data = l[mylist.index(m)]
    data_dict = data.text[112:-12]
    all_tables = json.loads(data_dict)
    return all_tables

def upgrade_downgrade_pandas(dic,tick):
    try:
        stock_upgrade_downgrade_history = dic['context']['dispatcher']['stores']['QuoteSummaryStore']['upgradeDowngradeHistory']['history']
        pd_stock_upgrade_downgrade_history = pd.DataFrame(stock_upgrade_downgrade_history)
        pd_stock_upgrade_downgrade_history['epochGradeDate'] = pd.to_datetime(pd_stock_upgrade_downgrade_history['epochGradeDate'],unit='s')
        pd_stock_upgrade_downgrade_history.columns.values[1] = 'date'
        pd_stock_upgrade_downgrade_history.insert(0,'stockTicker',tick)
        return pd_stock_upgrade_downgrade_history
    except Exception:
        print "error" + tick
        no_hist_symbol.append(tick)
        

def single_company_hist(tick):
    url = select_ticker(tick)
    soup = yahoo_finance_analyst_data(url)
    full_dict = grab_dict(soup)
    return upgrade_downgrade_pandas(full_dict,tick)

def list_of_all_stock_grading_history(good_list):
    
    all_stock_grading_history = []
    for ticker in good_list:
        if ticker not in no_hist_symbol:
            url = auto_symbols(ticker)
            soup = yahoo_finance_analyst_data(url)
            all_tables = grab_dict(soup)
            grading_history = upgrade_downgrade_pandas(all_tables,ticker)
            all_stock_grading_history.append(grading_history)
    concat_lists_to_pd = pd.concat(all_stock_grading_history)
    return concat_lists_to_pd


##############################################################################
##### Trading Data   #########################################################
##############################################################################

def get_stock_history(df):
    list_of_stock_histories = []
    for ticker in df['stockTicker'].unique():
        try:
            stock_df = pdr.get_data_yahoo(symbols=ticker, start=df.loc[df['stockTicker']==ticker].date.iloc[-1])
            stock_df['stockTicker'] = ticker
            stock_df.reset_index(inplace = True)
            stock_df['Percent_change'] =  stock_df.Close.pct_change()
            list_of_stock_histories.append(stock_df)
            print ticker
        except Exception: 
            print 'error @ ' + ticker
    concat_lists_to_pd = pd.concat(list_of_stock_histories)
    return concat_lists_to_pd
    

In [36]:
def business_days_and_datetimes(df1):
    df1['Date'] = pd.to_datetime(df1['Date'])
    df1['1_day_before_grade_Date'] = df1.loc[~df1['firm'].isnull()].Date.apply(lambda x: x + BDay(-1))
    df1['1_day_from_grade_Date'] = df1.loc[~df1['firm'].isnull()].Date.apply(lambda x: x + BDay(1))
    df1['30_days_from_grade_Date'] = df1.loc[~df1['firm'].isnull()].Date.apply(lambda x: x + BDay(22))
    df1['60_days_from_grade_Date'] = df1.loc[~df1['firm'].isnull()].Date.apply(lambda x: x + BDay(44))
    df1['90_days_from_grade_Date'] = df1.loc[~df1['firm'].isnull()].Date.apply(lambda x: x + BDay(66))

    df1['1_day_before_grade_Date'] =  pd.to_datetime(df1['1_day_before_grade_Date'])
    df1['1_day_from_grade_Date'] =  pd.to_datetime(df1['1_day_from_grade_Date'])
    df1['30_days_from_grade_Date'] =  pd.to_datetime(df1['30_days_from_grade_Date'])
    df1['60_days_from_grade_Date'] =  pd.to_datetime(df1['60_days_from_grade_Date'])
    df1['90_days_from_grade_Date'] =  pd.to_datetime(df1['90_days_from_grade_Date'])
    
    return df1

In [43]:
def merge_day_prices(df1):
    lst=[]
    unique_list = df1['stockTicker'].unique().tolist()
    list_of_dfs = [df1.loc[df1.stockTicker==word] for word in unique_list]
    for x in range(len(list_of_dfs)):
        df = list_of_dfs[x]
        
        tick = df5['stockTicker'].unique().tolist()[x]

    
        alldaysfrom = df.loc[df['Date'].isin(df['1_day_from_grade_Date'])]['Date']
        alldaysbefore = df.loc[df['Date'].isin(df['1_day_before_grade_Date'])]['Date']
        day30 = df.loc[df['Date'].isin(df['30_days_from_grade_Date'])]['Date']
        day60 = df.loc[df['Date'].isin(df['60_days_from_grade_Date'])]['Date']
        day90 = df.loc[df['Date'].isin(df['90_days_from_grade_Date'])]['Date']
        
        pricesfrom = df.loc[df['Date'].isin(df['1_day_from_grade_Date'])].Open
        pricesbefore = df.loc[df['Date'].isin(df['1_day_before_grade_Date'])].Open
        prices30 = df.loc[df['Date'].isin(df['30_days_from_grade_Date'])].Open
        prices60 = df.loc[df['Date'].isin(df['60_days_from_grade_Date'])].Open
        prices90 = df.loc[df['Date'].isin(df['90_days_from_grade_Date'])].Open

        allmerge = pd.DataFrame({'stockTicker':tick,
                                 '1_day_from_grade_Date':alldaysfrom,
                                 '1_day_from_grade_Date_Price':pricesfrom,
                                 '1_day_before_grade_Date':alldaysbefore,
                                 '1_day_before_grade_Date_Price':pricesbefore,
                                 '30_days_from_grade_Date':day30,
                                 '30_days_from_grade_Date_Price':prices30,
                                 '60_days_from_grade_Date':day60,
                                 '60_days_from_grade_Date_Price':prices60,
                                 '90_days_from_grade_Date':day90,
                                 '90_days_from_grade_Date_Price':prices90})
        
        lst.append(allmerge)
    concat = pd.concat(lst)
    day30sep = concat[['stockTicker','30_days_from_grade_Date','30_days_from_grade_Date_Price']]
    day30sep = day30sep.loc[~day30sep['30_days_from_grade_Date'].isnull()].drop_duplicates(subset=['30_days_from_grade_Date','30_days_from_grade_Date_Price'])

    day60sep = concat[['stockTicker','60_days_from_grade_Date','60_days_from_grade_Date_Price']]
    day60sep = day60sep.loc[~day60sep['60_days_from_grade_Date'].isnull()].drop_duplicates(subset=['60_days_from_grade_Date','60_days_from_grade_Date_Price'])

    day90sep = concat[['stockTicker','90_days_from_grade_Date','90_days_from_grade_Date_Price']]
    day90sep = day90sep.loc[~day90sep['90_days_from_grade_Date'].isnull()].drop_duplicates(subset=['90_days_from_grade_Date','90_days_from_grade_Date_Price'])

    day1bsep = concat[['stockTicker','1_day_before_grade_Date','1_day_before_grade_Date_Price']]
    day1bsep = day1bsep.loc[~day1bsep['1_day_before_grade_Date'].isnull()].drop_duplicates(subset=['1_day_before_grade_Date','1_day_before_grade_Date_Price'])

    day1asep = concat[['stockTicker','1_day_from_grade_Date','1_day_from_grade_Date_Price']]
    day1asep = day1asep.loc[~day1asep['1_day_from_grade_Date'].isnull()].drop_duplicates(subset=['1_day_from_grade_Date','1_day_from_grade_Date_Price'])
    
    day30 = pd.merge(df1,day30sep,how='outer',on=['stockTicker','30_days_from_grade_Date'])
    day6030 = pd.merge(day30,day60sep,how='outer',on=['stockTicker','60_days_from_grade_Date'])
    day906030 = pd.merge(day6030,day90sep,how='outer',on=['stockTicker','90_days_from_grade_Date'])
    day1906030 = pd.merge(day906030,day1bsep,how='outer', on=['stockTicker','1_day_before_grade_Date'])
    with_from_before_price = pd.merge(day1906030,day1asep,how='outer', on=['stockTicker','1_day_from_grade_Date'])
    
    with_from_before_price['percent30'] = 100 * (with_from_before_price['30_days_from_grade_Date_Price'] - with_from_before_price['Open']) / with_from_before_price['Open']
    with_from_before_price['percent60'] = 100 * (with_from_before_price['60_days_from_grade_Date_Price'] - with_from_before_price['Open']) / with_from_before_price['Open']
    with_from_before_price['percent90'] = 100 * (with_from_before_price['90_days_from_grade_Date_Price'] - with_from_before_price['Open']) / with_from_before_price['Open']
    with_from_before_price['percent1Before'] = 100 * (with_from_before_price['1_day_before_grade_Date_Price'] - with_from_before_price['Open']) / with_from_before_price['Open']
    with_from_before_price['percent1After'] = 100 * (with_from_before_price['1_day_from_grade_Date_Price'] - with_from_before_price['Open']) / with_from_before_price['Open']

    return with_from_before_price

In [47]:
def combine_toGrade(df1):
    
    Strong_Buy = ['Outperform','Mkt Outperformer','Recomm List','Market Outperform','Mkt Outperform','NT Strong Buy','Recommended List','LT Accumulate','Outperformer','Top Pick', 'Strong Buy','Outperf. Signif.','Sector Outperform','Conviction Buy']
    Buy = ['Buy','Overweight','Trading Buy','NT Accum/LT Buy', 'NT Ntrl/LT Buy','Long-Term Buy','Above Average','LT Attractive','LT Accum','LT Buy','Accumulate','NT Accum','NT Nuet/ LT Buy','Attractive','Positive','Long-term Buy','Add','NT Buy']
    Hold = ['Hold','Neutral','Peer perform','Equal-weight','Perform-In-Line','Perform In Line','In-line','Market Perform','Mkt Performer','Average','NT Neutral','Maintain Position','ST Neutral','Sector Perform','Mixed','Market Weight','Equal-weight','Perform','Sector Weight','Peer Perform','In-Line','Fair Value','Mkt Perform','Equal Weight']
    Underperform = ['Underperform', 'Underperformer','Below Average','Unattractive','Market Underperform','Cautious','Mkt Underperform','Sector Underperform']
    Sell = ['Sell','Underweight','Reduce','Negative','Trim']

    list_dict = {
    'Strong_buy':Strong_Buy,
    'Buy':Buy,
    'Hold':Hold,
    'Underperform':Underperform,
    'Sell':Sell
    }
    reversed_dict = {val: key for key in list_dict for val in list_dict[key]}
    df1['toGrade_combined'] = df1['toGrade'].map(reversed_dict)
    df1['fromGrade_combined'] = df1['fromGrade'].map(reversed_dict)
    return df1


## S&P 500 Dataset
* don't uncomment and run, this code creates the database, the database is already created through the file snp500merged


* snp500data gathers stock ticker analyst upgrade downgrade history, and is written into a csv called 'snp500analyst_history'
* snp500StockValueHistories takes the dates of all the stock data and returns the stock chart, and is written to the file 'snp500stockHistories'

* sfinal merges both of the two dataframes on 'stockTicker' and 'Date' and this dataframe is written to a csv called 'snp500merged'
    - Dates and Stock Prices are added 1 day before, 1 day after, 30 days after, 60 days after, and 90 days after
    - Percent Changes are added between the Open Price and the days added above

In [34]:
# snp500data = list_of_all_stock_grading_history(snp500tickers)
# snp500data.to_csv('snp500analyst_history')
# snp500StockValueHistories = get_stock_history(snp_500_data_df)
# snp500StockValueHistories.to_csv('snp500stockHistories')
# snp_500_analyst_history_df = pd.read_csv('snp500analyst_history')
# snp_500_stock_history_df = pd.read_csv('snp500stockHistories')
# snp500_stock_and_updown_hist_merged_df = snp_500_stock_history_df.merge(snp_500_analyst_history_df,how='left',left_on= ['Date','stockTicker'], right_on=['date','stockTicker'])
# cols = list(snp500_stock_and_updown_hist_merged_df.columns.values)
# snp500_stock_and_updown_hist_merged_df = snp500_stock_and_updown_hist_merged_df.drop('Unnamed: 0_y',1)
# snp500_stock_and_updown_hist_merged_df = snp500_stock_and_updown_hist_merged_df[['stockTicker','Date','firm','action','fromGrade','toGrade','Open', 'High','Low','Close','Adj Close','Volume']]
# df5 = snp500_stock_and_updown_hist_merged_df.copy()
# business_days = business_days_and_datetimes(df5)
# with_from_before_price = merge_day_prices(list_of_dfs,business_days)
# finaldf = combine_toGrade(with_from_before_price)
# finaldf.to_csv('snp500merged')
# finaldf.drop('Unnamed: 0',1)
stock_analyst_df = pd.read_csv('snp500merged')

* Removed day high, day low, and adj close

In [48]:
finaldf = finaldf[['stockTicker',
 'Date',
 'Open',
 'Close',
 'Volume',
 'firm',
 'action',
 'fromGrade',
 'fromGrade_combined',          
 'toGrade',
 'toGrade_combined',
 '1_day_before_grade_Date',
 '1_day_before_grade_Date_Price',
 'percent1Before',
 '1_day_from_grade_Date',
 '1_day_from_grade_Date_Price',
 'percent1After',
 '30_days_from_grade_Date',
 '30_days_from_grade_Date_Price',
 'percent30',
 '60_days_from_grade_Date',
 '60_days_from_grade_Date_Price',
 'percent60',
 '90_days_from_grade_Date',
 '90_days_from_grade_Date_Price',
 'percent90']]

In [58]:
finaldf.to_csv('snp500merged')

## All Stock Ticker Dataset

In [ ]:
#all_ticker_data = list_of_all_stock_grading_history(entire_ticker_list)
#all_ticker_data.to_csv('entire_ticker_analyst_history')
#all_ticker_data.head()

In [4]:
all_ticker_data = pd.read_csv('C:\Users\sarab\Desktop\git_projects\stock_analyst_grade_history\entire_ticker_analyst_history')
snp_500_data_df = pd.read_csv('C:\Users\sarab\Desktop\git_projects\stock_analyst_grade_history\snp500analyst_history')

### Single Company Lookup by Ticker

In [ ]:
df = single_company_hist(raw_input('Please Enter Ticker Value').upper())

In [54]:
snp500StockValueHistories.head()

,Date,Open,High,Low,Close,Adj Close,Volume,stockTicker,Percent_change
0,2012-02-09,87.540001,88.180000,87.459999,88.019997,75.877464,3031600.0,MMM,NaN
1,2012-02-10,87.199997,87.559998,86.750000,87.139999,75.118858,3085700.0,MMM,-0.009998
2,2012-02-13,87.570000,88.089996,87.160004,88.029999,75.886070,2530600.0,MMM,0.010213
3,2012-02-14,87.599998,88.000000,87.220001,87.989998,75.851593,2850300.0,MMM,-0.000454
4,2012-02-15,87.419998,87.720001,86.750000,87.010002,75.513138,3647000.0,MMM,-0.011138


## SNP500 Stock Value Histories

In [16]:
#snp500StockValueHistories = get_stock_history(snp_500_data_df)
#snp500StockValueHistories.to_csv('snp500stockHistories')

## Stock History and Analyst History Merged Data